# Modeling
In this notebook, we'll be modeling the data we've previously prepared. Out notebook will be laid out as follows:

1. Model Selection & Generation
2. Hyperparameter Optimization
3. Fine-Tuning (if needed)
4. Reporting Best Model(s) + Settings
5. Interpretation
6. Conclusion

Our eventual goal here is two-fold:

1. Accurately [and fairly] model the diabetes dataset
2. Interpret the results to find something worth recommending to those wanting to reduce risk of diabetes. This can be via LIME/SHAP (i.e. some interpretable model that approximates the neural network) or via analyzing a more simple model's structure (i.e. regression coefficients, random forest decision boundaries)

In [2]:
# Environment Setup
from utils.model import *
from utils.dataset import *

***
## Model Selection & Generation

In [4]:
# generate lookup for models
models = {
    # "tree": TreeClassifier(target="diabetes", path="../datasets/processed.parquet"),
    "ffnn": MLPClassifier(target="diabetes", path="../datasets/processed.parquet")
}

# manual search
models["ffnn"].set_hyperparams({
    "learning_rate": .0001,
    "batch_size": 128,
    "num_hidden": 1,
    "hidden_size": 4096,
    "num_epochs": 50,
    "classify_fn": "softmax"
})

# train & test basic model
for mt, model in models.items():
    # attempt to load, else train and test
    # if not model.load_model():
    model.train_model(verbose=1)
    model.test_model()

<Train-Test Split Report>
Train: 512887 obs, 170945 no diabetes [0], 171126 pre-diabetes [1], 170816 diabetes [2]
Test: 128222 obs, 42758 no diabetes [0], 42577 pre-diabetes [1], 42887 diabetes [2]


100%|██████████| 4007/4007 [00:13<00:00, 289.12it/s]


Epoch 1/50, Loss: 1.0077912219450722


100%|██████████| 4007/4007 [00:13<00:00, 287.87it/s]


Epoch 2/50, Loss: 0.9904404963553324


...

Epoch 48/50, Loss: 0.8935286043765451


100%|██████████| 4007/4007 [00:13<00:00, 295.86it/s]


Epoch 49/50, Loss: 0.8889047434748275


100%|██████████| 4007/4007 [00:13<00:00, 287.51it/s]


Epoch 50/50, Loss: 0.8990425235736629

<Test Report>
Precision: [no diabetes] 0.6628577131309099, [pre-diabetes] 0.5269758437841933, [diabetes] 0.5684811778319423
Recall: [no diabetes] 0.6213574068010664, [pre-diabetes] 0.5882048993588087, [diabetes] 0.5383915871942546
F1-Score: [no diabetes] 0.6414370043095643, [pre-diabetes] 0.5559094793620493, [diabetes] 0.5530273998850355
Support: [no diabetes] 42758, [pre-diabetes] 42577, [diabetes] 42887
Accuracy: 58.2599%


***
## Hyperparameter Optimization

In [ ]:
# optimize hyperparams
optimizer_results = {model_type: model.optimize_hyperparams(kfold=2) for model_type, model in models.items()}
print(optimizer_results)

<Grid-Search>
Testing 7 combinations WITHOUT cross-validation


IndexError: tuple index out of range

***
## Fine-Tuning + Other Adjustments

***
## Best Model Report

***
## Interpretation

***
## Conclusion